In [ ]:
!pip install adversarial-robustness-toolbox
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from datetime import datetime 
from tqdm.notebook import tqdm 
import statistics
from math import log10
import struct
from random import randrange
import multiprocessing
import concurrent.futures
import time

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from sklearn.preprocessing import QuantileTransformer, PowerTransformer

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib 

fr_global = 100

     |████████████████████████████████| 1.2 MB 12.9 MB/s 
     |████████████████████████████████| 3.3 MB 51.1 MB/s 
     |████████████████████████████████| 34.5 MB 1.3 MB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [ ]:
train_size = 10000

filename = '../dataset/purchase_member.csv'
# filename = '/content/purchase_member.csv'
df_train = pd.read_csv (filename, header=None,   sep='\t', encoding='utf-8') 

filename = '../dataset/purchase_non_member.csv'
# filename = '/content/purchase_non_member.csv'
df_test = pd.read_csv (filename,  header=None,  sep='\t', encoding='utf-8') 

print(df_train.shape, df_test.shape, f'{len(df_train)/(len(df_train)+len(df_test)):0.2f}')


#separating label from data and converting dataframe into Torch Tensor
col_0 = df_train.columns[0] # 1st column is label; 
col_rest = df_train.columns[1:] # rests are data
X_train = torch.tensor(df_train[col_rest].values, dtype=torch.float32) 
y_train = torch.tensor(df_train[col_0].values) # y is row vector here

print(f'-'*30, 'train', f'-'*30)
display(X_train.size(), y_train.size())
display(X_train, y_train)


(10000, 601) (32254, 601) 0.24
------------------------------ train ------------------------------


torch.Size([10000, 600])

torch.Size([10000])

tensor([[0., 1., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

tensor([54, 36, 97,  ..., 97, 71,  7])

In [ ]:
'''
Approximate model ...
'''

# any element(float point) of tensor is represented by 32 bits
def fault_injection_FP(dec_list):
    
    fault_rate = fr_global
    
    output_list = []
    
    for dec in dec_list:

        a = ''.join(bin(c).replace('0b', '').rjust(8, '0') for c in struct.pack('!f', dec))        

        #generate 32-bit binary mask for given fault rate
        mask_bin = []
        #Keep the most significant bit (sign bit) and 5 least significant bits unchanged
        mask_bin = ['0'] + ['1' if randrange(1000000) < fault_rate else '0' for i in range(26)] + ['0' for i in range(5)]    

        a_faulty = list('00000000000000000000000000000000')
        a_bin = list(a)
        for i in range(32):
            a_faulty[i] = str(int(a_bin[i])^int(mask_bin[i])) 
        aa = ''.join(a_faulty) 
        f = struct.unpack('!f',struct.pack('!I', int(aa, 2)))[0]
        
        output_list.append(f)
    
    return output_list
          

def fault_injection_FC_layer(x, layer, status):   
    
    #set fault_rate = 0 during train time.
    #set fault_rates = 1, 10, 100, 1000, 10000, 100000, 1000000, one by one, during test time
    #these fault rates corresponse to actual fault rate of 10^-6, 10^-5, 10^-4, 10^-3, 10^-2, 10^-1, 10^-0 
    #----------------------------------------------------------------------------------------------------
    fault_rate = fr_global
    
    if (status == 'ON'):
      # print(f'\tfault_rate = {fault_rate/1000000:.0e};   {layer} is {status}')

      x_dim = list(x.size())
      n_row = x_dim[0]
      n_column = x_dim[1]
      
      x_list = [element.item() for element in x.flatten()]

      n_process = multiprocessing.cpu_count()
      k, m = divmod(len(x_list), n_process)
      x_list_sublist = list((x_list[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n_process)))

      with concurrent.futures.ProcessPoolExecutor() as executor:
          output_list_sublist = executor.map(fault_injection_FP, x_list_sublist)

      output_flat_list = [item for sublist in output_list_sublist for item in sublist] 

      shape = (n_row, n_column)
      output_array = np.array(output_flat_list)
      output_array = output_array.reshape(shape )

      output_tensor = torch.from_numpy(output_array).float()
      x.data = output_tensor.data
 
    return x


In [ ]:

# PurchaseClassifier_Approx model

class PurchaseClassifier_Approx(nn.Module):

    def __init__(self, num_features = 600, num_classes=100):
        super(PurchaseClassifier_Approx, self).__init__() 
        self.fc1 = nn.Linear(num_features,1024)
        self.fc2 = nn.Linear(1024,512)
        self.fc3 = nn.Linear(512,256)
        self.fc4 = nn.Linear(256,128)
        self.fc5 = nn.Linear(128,num_classes)
        self.relu = nn.Tanh()

    def forward(self, x):
       #classifier
        x = self.fc1(x)
        x = fault_injection_FC_layer(x, layer='FC-1', status='ON')
        x = self.relu(x)

        x = self.fc2(x)
        x = fault_injection_FC_layer(x, layer='FC-2', status='ON') 
        x = self.relu(x)

        x = self.fc3(x) 
        x = fault_injection_FC_layer(x, layer='FC-3', status='ON')
        x = self.relu(x)

        x = self.fc4(x)
        # x = fault_injection_FC_layer(x, layer='FC-4', status='ON')
        x = self.relu(x)

        x = self.fc5(x)

        # #sigmoid returns a value between 0 and 1, used for binary classification
        # prob = torch.sigmoid(x)   
        
        logits = x
        return logits




target_model_vos = PurchaseClassifier_Approx()

model_path = '../dataset/Customer100_10k_train.pth'
# model_path = '/content/Customer100_10k_train.pth'
target_model_vos.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
target_model_vos.eval()


PurchaseClassifier_Approx(
  (fc1): Linear(in_features=600, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=100, bias=True)
  (relu): Tanh()
)

In [ ]:
import time

epoch = [500]
exetime = []
fr_global = 1

print(f'\tfault_rate = {fr_global/10000000:.0e}...')

one_sample = torch.tensor(df_train.head(1)[col_rest].values, dtype=torch.float32) 

for e in epoch:
  start_time = time.time()
  for i in range(e):
    _ = target_model_vos(one_sample)

  time_one_sample = (time.time() - start_time)*1000000
  exetime.append(time_one_sample)

print(f"One inference time: --- {exetime} microseconds")

 
 

	fault_rate = 1e-07...
One inference time: --- [115805553.67469788] microseconds


In [ ]:
t = [f'& {round(t/1000000, 2)} &' for t in exetime]
print(t)

['& 0.55 &', '& 10.7 &', '& 21.24 &', '& 32.03 &', '& 42.96 &', '& 54.53 &', '& 283.94 &']


In [ ]:
def get_accuracy(model, data_loader):
    '''
    Function for computing the accuracy of the predictions over the entire data_loader
    '''
    
    correct_pred = 0 
    n = 0
    
    with torch.no_grad():
        model.eval()
        for X, y_true in data_loader:

            y_hat = model(X)
            _, predicted_labels = torch.max(y_hat, 1)

            n += y_true.size(0)
            correct_pred += (predicted_labels == y_true).sum()

    return correct_pred.float() / n


In [ ]:
'''
MIA with Logit
'''

def mia_with_logit(y_hat):

  df_logit = pd.DataFrame(y_hat.detach().numpy())
  scaler = StandardScaler() #94 70
  df_logit  = pd.DataFrame(scaler.fit_transform(df_logit))
  df_logit = df_logit.fillna(0)

  y_test2_np = np.array([int(1) for i in range(len(df_logit))]) 

  n_feature = len(df_logit.columns) 
  attacker_mlp_logit = MLPClassifier(hidden_layer_sizes=(n_feature, n_feature, 50, 25), activation='relu', max_iter = 2000, random_state = 1)
  attacker_mlp_logit = joblib.load('../dataset/attacker_mlp_logit.pkl')

  y_pred_np = attacker_mlp_logit.predict(df_logit)
  accuracy  = round(np.mean(y_pred_np == y_test2_np), 2)
  # print(f'MIA with Logit: {accuracy}')
  return accuracy


In [ ]:
'''
MIA with Probability 
'''

def mia_with_probability(y_hat):

  logit = y_hat
  X_mem_prob = torch.sigmoid(logit)
  X_test_attacker_df = pd.DataFrame(X_mem_prob.detach().numpy())

  scaler = StandardScaler() #94 70
  X_test_attacker_df  = pd.DataFrame(scaler.fit_transform(X_test_attacker_df))
  X_test_attacker_df = X_test_attacker_df.fillna(0)

  y_test2_np = np.array([int(1) for i in range(len(X_test_attacker_df))]) 

  n_feature = len(X_test_attacker_df.columns) 
  attacker_mlp_prob = MLPClassifier(hidden_layer_sizes=(n_feature, n_feature, 50, 25), activation='relu', max_iter = 2000, random_state = 1)
  attacker_mlp_prob = joblib.load('../dataset/attacker_mlp_prob.pkl')

  y_pred_np = attacker_mlp_prob.predict(X_test_attacker_df)
  accuracy  = round(np.mean(y_pred_np == y_test2_np), 2)
  # print(f'MIA with Probability:  {accuracy}')
  return accuracy



In [ ]:
'''
MIA with Probability & Loss

'''

def myCustomLoss(my_outputs, my_labels):
  #specifying the batch size
  my_batch_size = my_outputs.size()[0] 
  #calculating the log of softmax values           
  my_outputs = F.log_softmax(my_outputs, dim=1)  
  #selecting the values that correspond to labels
  my_outputs = my_outputs[range(my_batch_size), my_labels] 
  #returning the results
  return my_outputs

def mia_with_probability_loss(y_hat, y_test):

  logit = y_hat
  X_mem_nonmem_prob = torch.sigmoid(logit)
  df_tmp = pd.DataFrame(X_mem_nonmem_prob.detach().numpy())

  loss = myCustomLoss(logit, y_test)
  df_loss = pd.DataFrame(loss.detach().numpy())
  # scaler = StandardScaler() #94 70
  scaler = RobustScaler() #94 70
  df_loss  = pd.DataFrame(scaler.fit_transform(df_loss))

  X_test_attacker_df = pd.concat([df_tmp, df_loss], ignore_index=True, axis=1) #axis=1 merges 2 dataframes side by side
  X_test_attacker_df = X_test_attacker_df.fillna(0)

  y_test2_np = np.array([int(1) for i in range(len(X_test_attacker_df))]) 

  n_feature = len(X_test_attacker_df.columns) 
  attacker_mlp_prob_loss = MLPClassifier(hidden_layer_sizes=(n_feature, n_feature, 50, 25), activation='relu', max_iter = 2000, random_state = 1)
  attacker_mlp_prob_loss = joblib.load('../dataset/attacker_mlp_prob_loss.pkl')

  y_pred_np = attacker_mlp_prob_loss.predict(X_test_attacker_df)
  accuracy  = round(np.mean(y_pred_np == y_test2_np), 2)
  # print(f'MIA with Probability & Loss:  {accuracy}')
  return accuracy



In [ ]:
'''
Target Model Test accuracy
'''
#separating label from data and converting dataframe into Torch Tensor
col_0 = df_test.columns[0] # 1st column is label; 
col_rest = df_test.columns[1:] # rests are data
X_test = torch.tensor(df_test.head(5000)[col_rest].values, dtype=torch.float32) 
y_test = torch.tensor(df_test.head(5000)[col_0].values) # y is row vector here

print(f'-'*30, 'test', f'-'*30)


baseline_acc_mean = [] # target model test accuracy under VOS
baseline_acc_std = []

MIA_logit_mean = []
MIA_logit_std = []

MIA_prob_mean = []
MIA_prob_std = []

MIA_prob_loss_mean = []
MIA_prob_loss_std = []

fault_rates = [1, 10, 100, 1000]

for fr in fault_rates:

  fr_global = fr

  print(f'\tfault_rate = {fr_global/1000000:.0e}...')
  print(f'-'*60)

  baseline_tmp = []
  MIA_logit_tmp = []
  MIA_prob_tmp = []
  MIA_prob_loss_tmp = []

  for i in range(50):
    y_hat = target_model_vos(X_test)
    _, predicted_labels = torch.max(y_hat, 1)
    n = y_test.size(0)
    correct_pred = (predicted_labels == y_test).sum()
    test_acc = correct_pred.float() / n
    test_acc  = round(test_acc.item(), 2)

    baseline_tmp.append(test_acc)
    MIA_logit_tmp.append(mia_with_logit(y_hat))
    MIA_prob_tmp.append(mia_with_probability(y_hat))
    MIA_prob_loss_tmp.append(mia_with_probability_loss(y_hat, y_test))

  baseline_acc_mean.append(statistics.mean(baseline_tmp))
  baseline_acc_std.append(statistics.stdev(baseline_tmp))

  MIA_logit_mean.append(statistics.mean(MIA_logit_tmp))
  MIA_logit_std.append(statistics.stdev(MIA_logit_tmp))

  MIA_prob_mean.append(statistics.mean(MIA_prob_tmp))
  MIA_prob_std.append(statistics.stdev(MIA_prob_tmp))

  MIA_prob_loss_mean.append(statistics.mean(MIA_prob_loss_tmp))
  MIA_prob_loss_std.append(statistics.stdev(MIA_prob_loss_tmp))






------------------------------ test ------------------------------
	fault_rate = 1e-06...
------------------------------------------------------------
	fault_rate = 1e-06;   FC-1 is ON
	fault_rate = 1e-06;   FC-2 is ON
	fault_rate = 1e-06;   FC-3 is ON
	fault_rate = 1e-06;   FC-1 is ON
	fault_rate = 1e-06;   FC-2 is ON
	fault_rate = 1e-06;   FC-3 is ON
	fault_rate = 1e-06;   FC-1 is ON
	fault_rate = 1e-06;   FC-2 is ON
	fault_rate = 1e-06;   FC-3 is ON
	fault_rate = 1e-06;   FC-1 is ON
	fault_rate = 1e-06;   FC-2 is ON
	fault_rate = 1e-06;   FC-3 is ON
	fault_rate = 1e-06;   FC-1 is ON
	fault_rate = 1e-06;   FC-2 is ON
	fault_rate = 1e-06;   FC-3 is ON
	fault_rate = 1e-06;   FC-1 is ON
	fault_rate = 1e-06;   FC-2 is ON
	fault_rate = 1e-06;   FC-3 is ON
	fault_rate = 1e-06;   FC-1 is ON
	fault_rate = 1e-06;   FC-2 is ON
	fault_rate = 1e-06;   FC-3 is ON
	fault_rate = 1e-06;   FC-1 is ON
	fault_rate = 1e-06;   FC-2 is ON
	fault_rate = 1e-06;   FC-3 is ON
	fault_rate = 1e-06;   FC-1 is ON

In [ ]:
print(f'baseline_acc_mean: {baseline_acc_mean}')
print(f'baseline_acc_std: {baseline_acc_std}')
print()

print(f'MIA_logit_mean: {MIA_logit_mean}')
print(f'MIA_logit_std: {MIA_logit_std}')
print()

print(f'MIA_prob_mean: {MIA_prob_mean}')
print(f'MIA_prob_std: {MIA_prob_std}')
print()

print(f'MIA_prob_loss_mean: {MIA_prob_loss_mean}')
print(f'MIA_prob_loss_std: {MIA_prob_loss_std}')


baseline_acc_mean: [0.78, 0.77, 0.75, 0.547]
baseline_acc_std: [0.0, 0.0, 0.0, 0.004830458915396484]

MIA_logit_mean: [0.35, 0.35, 0.338, 0.245]
MIA_logit_std: [0.0, 0.0, 0.004216370213557843, 0.0052704627669473035]

MIA_prob_mean: [0.14, 0.14, 0.166, 0.393]
MIA_prob_std: [0.0, 0.0, 0.005163977794943227, 0.004830458915396484]

MIA_prob_loss_mean: [0.36, 0.36, 0.383, 0.555]
MIA_prob_loss_std: [0.0, 0.0, 0.004830458915396484, 0.0052704627669473035]
